In [29]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


import numpy as np

from preparation_funcs import time_based_split, apply_ce_encoding, apply_feature_scaling, generate_nonlinear_features
from choose_models_funcs import evaluate_models, custom_gini, evaluate_final_model
from custom_three_models import CustomDecisionTreeClassifier, CustomExtraTreesClassifier, CustomRandomForestClassifier, CustomGBDTClassifier

# Data loading

In [3]:
df = pd.read_csv(f"data/training.csv")

# 1. Подготовка данных

In [4]:
train, validation, test = time_based_split(df, "PurchDate")

*В качестве метода кодирования выбран CountEncoder в связи с большим количеством категориальных переменных, потенциально большим количеством категорий и высокой вероятностью уникальных категорий в тесте и валидационной выборке*

In [5]:
train_encoded, val_encoded, test_encoded = apply_ce_encoding(train, validation, test)

*Заполняем пропуски, преобразуем дату в timestamp и после стандартизируем все числовые столбцы, кроме целевого (он бинарный)*

In [107]:
train_scaled, val_scaled, test_scaled = apply_feature_scaling(train_encoded, val_encoded, test_encoded, columns = df.drop(["IsBadBuy"], axis=1).columns)

# 2-9. Обучение моделей

## custom. DecisionTreeClassifier (№ 3), RandomForestClassifier (№ 5), GBDT classifier (№ 6) и ExtraTreesClassifier (№ 9)

In [72]:
models_to_test_custom = [
    CustomDecisionTreeClassifier(),
    CustomRandomForestClassifier(random_state=42),
    CustomGBDTClassifier(random_state=42),
    CustomExtraTreesClassifier()
]

In [ ]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test_custom
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,CustomDecisionTreeClassifier,CustomRandomForestClassifier,CustomGBDTClassifier,CustomExtraTreesClassifier
Train,1.0,0.925788,0.503882,1.0
Validation,0.213597,0.375774,0.475597,0.179431


## sklearn. DecisionTreeClassifier (№ 4) и RandomForestClassifier

### Из коробки

In [105]:
models_to_test = [
    DecisionTreeClassifier(max_depth=100),
    RandomForestClassifier(random_state=42)
]

In [106]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,DecisionTreeClassifier,RandomForestClassifier
Train,1.0,1.0
Validation,0.187671,0.440653


* Разницу между кастомным и sklearn DecisionTreeClassifier можно объяснить более сложными настройками в sklearn версии и особенностями реализации сплита, который в sklearn более сложный, а также в связи с особенностями перебора признаков

### Консервативный тюнинг

In [68]:
models_to_test = [
    DecisionTreeClassifier(max_depth=5, min_samples_split=20, min_samples_leaf=10, max_features='sqrt', random_state=42),
    RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=10, min_samples_leaf=5, max_features='sqrt', random_state=42)
]

In [69]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,DecisionTreeClassifier,RandomForestClassifier
Train,0.479806,0.648301
Validation,0.38471,0.475752


## LightGBM, Catboost, XGBoost (№ 7)

### Из коробки

In [17]:
models_to_test = [
    XGBClassifier(random_state = 42, verbosity=0),
    LGBMClassifier(random_state = 42, verbose=0),
    CatBoostClassifier(random_seed = 42, verbose=0)
]

In [18]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,XGBClassifier,LGBMClassifier,CatBoostClassifier
Train,0.972177,0.858595,0.856114
Validation,0.442063,0.460408,0.462371


### Консервативный тюнинг

In [26]:
models_to_test = [
    XGBClassifier(
    # Основные параметры
    n_estimators=400,          # Ставим большое число, но используем early stopping
    learning_rate=0.02,         # Небольшая скорость обучения
    
    # Параметры для контроля переобучения
    max_depth=3,                # Неглубокие деревья. Главный параметр для контроля сложности модели
    min_child_weight=5,
    subsample=0.7,              # Использовать 70% данных для каждого дерева
    colsample_bytree=0.7,       # Использовать 70% признаков для каждого дерева
    gamma=2,                    # Параметр, затрудняющий создание новых ветвей
    reg_alpha=2,              # L1 регуляризация
    reg_lambda=2,             # L2 регуляризация

    # Технические параметры
    random_state=42,
    verbosity=0,
    use_label_encoder=False,
    eval_metric='logloss'
),
    LGBMClassifier( 
    # Основные параметры
    n_estimators=400,
    learning_rate=0.02,
    
    # Параметры для контроля переобучения
    num_leaves=15,              # Ограничиваем сложность. Главный параметр для контроля сложности модели
    max_depth=5,                # Дополнительное ограничение глубины
    min_child_samples=30,
    subsample=0.65,
    colsample_bytree=0.65,
    reg_alpha=5,
    reg_lambda=5,

    # Технические параметры
    random_state=42,
    verbose=-1,
    objective='binary',
    metric='auc'
),
    CatBoostClassifier(
    # Основные параметры
    iterations=500,
    learning_rate=0.02,
    
    # Параметры для контроля переобучения
    depth=4,      
    min_data_in_leaf=20,              
    subsample=0.7,
    colsample_bylevel=0.7,
    l2_leaf_reg=7,              

    # Технические параметры
    random_seed=42,
    verbose=0,
    auto_class_weights='Balanced' # Полезно для несбалансированных выборок
)
]

In [27]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,XGBClassifier,LGBMClassifier,CatBoostClassifier
Train,0.577051,0.624689,0.574714
Validation,0.493035,0.493343,0.49436


* LGBMClassifier требует более агрессивного тюнинга в связи со стратегией роста leaf-wise.
* Все модели показывают почти идентичное качество оценка на валидационном сете, что говорит о достижении потолка в качестве классификации с данным типом моделей с учетом консервативной стратегии сдерживания переобучения.


### Дивергентный тюнинг

In [99]:
models_to_test = [
    XGBClassifier(**{
    # --- Особенность: DART ---
    'booster': 'dart',
    'rate_drop': 0.15,           # Умеренная вероятность "выключения" дерева
    'skip_drop': 0.5,
    
    # --- Консервативная основа ---
    'n_estimators': 500,         # DART требует больше итераций
    'learning_rate': 0.02,       # Низкая скорость обучения
    'max_depth': 3,              # неглубокие деревья
    'min_child_weight': 5,       # Сильная регуляризация
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'gamma': 2,                  # Дополнительная регуляризация
    'reg_alpha': 2,
    'reg_lambda': 2,

    # --- Технические параметры ---
    'random_state': 42,
    'verbosity': 0,
    'eval_metric': 'logloss'
    }),
    LGBMClassifier( **{
    # --- Особенность: Линейные деревья ---
    'linear_tree': True,

    # --- Консервативная основа ---
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 400,
    'learning_rate': 0.02,
    'num_leaves': 15,            
    'max_depth': 5,             
    'min_child_samples': 30,
    'feature_fraction': 0.65,
    'bagging_fraction': 0.65,
    'reg_alpha': 5,
    'reg_lambda': 5,
    
    # --- Технические параметры ---
    'random_state': 42,
    'verbose': -1,
    'metric': 'auc'
})
]

In [100]:
evaluate_models(
    train_df=train_scaled, 
    val_df=val_scaled, 
    target_column='IsBadBuy', 
    models=models_to_test
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,XGBClassifier,LGBMClassifier
Train,0.559453,0.71218
Validation,0.489949,0.424002


In [94]:
train_cat, val_cat, test_cat = apply_feature_scaling(train, validation, test)
categorical_features_names = list(train_cat.select_dtypes(include=['object', 'category']).columns)
for col in categorical_features_names:
    train_cat[col].fillna('missing', inplace=True)
    val_cat[col].fillna('missing', inplace=True)
    test_cat[col].fillna('missing', inplace=True)

/tmp/ipykernel_6142/4249462042.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_cat[col].fillna('missing', inplace=True)
/tmp/ipykernel_6142/4249462042.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [101]:
evaluate_models(
    train_df=train_cat, 
    val_df=val_cat, 
    target_column='IsBadBuy', 
    models=[CatBoostClassifier(**{
    # --- Особенность: Обработка категорий ---
    'cat_features': categorical_features_names, 

    # --- Консервативная основа ---
    'iterations': 500,            
    'learning_rate': 0.02,
    'depth': 4,                 
    'l2_leaf_reg': 7,             
    'subsample': 0.7,  
    "colsample_bylevel":0.7, 
    'min_data_in_leaf': 20,

    # --- Технические параметры ---
    'random_seed': 42,
    'verbose': 0,
    'auto_class_weights': 'Balanced'
})]
)

--- Оценка набора признаков: 'Full Features' (33 признаков) ---

Результаты для 'Full Features' (метрика: gini_score_sklearn):


,CatBoostClassifier
Train,0.566969
Validation,0.495577


**Консервативные настройки не сильно отличаются от дивергентныех для XGBClassifier (dart - это способ регуляризации, т.ч. на текущем наборе данных роли не играет) и CatBoostClassifier (предварительная обработка категориальных переменных была не хуже автоматической). В случае LGBMClassifier наблюдается большее переобучение, что связано с тем, что linear_tree делает модель слишком сложной и склонной к переобучению**

# Оценка финальной модели

In [102]:
best = CatBoostClassifier(**{
    # --- Особенность: Обработка категорий ---
    'cat_features': categorical_features_names, 

    # --- Консервативная основа ---
    'iterations': 500,            
    'learning_rate': 0.02,
    'depth': 4,                 
    'l2_leaf_reg': 7,             
    'subsample': 0.7,  
    "colsample_bylevel":0.7, 
    'min_data_in_leaf': 20,

    # --- Технические параметры ---
    'random_seed': 42,
    'verbose': 0,
    'auto_class_weights': 'Balanced'
})

In [104]:
evaluate_final_model(best, train_cat, val_cat, test_cat, 'IsBadBuy')

Обучение финальной модели 'CatBoostClassifier'...

--- Финальная оценка Gini ---


,Gini
Dataset,
Train,0.567
Validation,0.496
Test,0.491


**Можно говорить о том, что значительного переобучения нет, т.к. разница между оценками для трейна, валидации и теста небольшая**